In [1]:
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import sqlite3
import time
import random

data
map 1 = 0/1
map 2 = 4/5
map 3 = 6/7

In [2]:
def get_totals(cell):
    return cell.split()[0]

def get_totals_d(cell):
    return cell.split()[1]

def format_dataframe(dataframes, map_num):
    """
    unduplicates data

    :param dataframes: dataframes returned using the pandas read_html function on the page source
    :param map_num: indexed starting at 1 for which map is being played in a series
    :return: formatted df
    """

    if map_num == 1:
        formatting_df = pd.concat(dataframes[:2])
    elif map_num == 2:
        formatting_df = pd.concat([dataframes[4], dataframes[5]])


    elif map_num == 3:
        try:
            formatting_df = pd.concat([dataframes[6], dataframes[7]])
        except IndexError as x:
            print('map 3 link not working')
            return



    formatting_df = formatting_df.reset_index()
    formatting_df = formatting_df.drop(columns=['index', 'Unnamed: 1'])

    formatting_df.iloc[:, 1:4] = formatting_df.iloc[:, 1:4].applymap(get_totals)
    formatting_df.iloc[:, 5:] = formatting_df.iloc[:, 5:].applymap(get_totals)
    formatting_df['PLAYER'] = formatting_df['Unnamed: 0'].apply(lambda x: x.split()[0])
    formatting_df['TEAM'] = formatting_df['Unnamed: 0'].apply(lambda x: x.split()[1])

    formatting_df = formatting_df.drop(columns=['Unnamed: 0'])

    formatting_df.iloc[:, 3] = formatting_df.iloc[:, 3].apply(get_totals_d)

    return formatting_df


def get_match_results_data(source, link):
    """
    :param source: pagesource
    :param link: link for particular match results page
    :return: multiple dataframes of match results, names for images/agent names
    """
    dfs = pd.read_html(source)

    img_elements = driver.find_elements(By.TAG_NAME, value='img')

    # Loop through each image element and extract the "alt" text
    alt_texts = []
    for img_element in img_elements:
        alt_text = img_element.get_attribute('alt')
        if alt_text:
            alt_texts.append(alt_text)


    return dfs, alt_texts

team information
0/1 = map 1 = 0
4/5 = map 2 = 1
8/9 = map 3 = 2


In [3]:
def get_data(driver, map_num, map_name):
    dfs = pd.read_html(driver.page_source)
    df1 = format_dataframe(dfs, map_num)

    #skip if no table is provided
    if df1 is None:
        return

    #define element needed to be searched for team information depending on map number
    if map_num == 1:
        team_nums = [0,1]
    elif map_num == 2:
        team_nums = [4,5]
    elif map_num == 3:
        team_nums = [8,9]

    #add agents
    dfs, alt_texts = get_match_results_data(driver.page_source, links[0])
    df1['AGENT'] = alt_texts[2:12]

    # Get Time/Teams/Map for creating a primary key
    time_of_map = driver.find_element(By.CLASS_NAME, value='moment-tz-convert')
    time_of_map = time_of_map.get_attribute('data-utc-ts')

    # map_name = driver.find_element(By.CLASS_NAME, value='map')
    # print(map_name.text)
    # map_name = map_name.text.split('\n')[0]
    df1['map'] = map_name


    #get rounds won and team names
    team_information = driver.find_elements(By.CLASS_NAME, value='vm-stats-game-header')[map_num-1]
    right_team = team_information.find_element(By.CLASS_NAME, value='team.mod-right')
    left_team = team_information.find_element(By.CLASS_NAME, value='team')

    formatted_team_information = right_team.text.split('\n')

    away_team_information = formatted_team_information[0]
    away_rounds_won = formatted_team_information[-1]
    df1['rounds_won'] = 0
    df1['rounds_won'].iloc[5:] = away_rounds_won

    formatted_team_information_home = left_team.text.split('\n')

    home_team_information = formatted_team_information_home[-2]
    home_rounds_won = formatted_team_information_home[0]
    df1['rounds_won'].iloc[:5] = home_rounds_won

    #create primary key
    df1['match_id'] = home_team_information + ' ' + away_team_information + ' ' + map_name + ' ' + time_of_map

    return df1

In [4]:
# create a function to import pandas dataframes into the database
def import_df_to_sqlite(df, table_name, conn):
    df.to_sql(name=table_name, con=conn, if_exists='append', index=False)

def dataframes_to_sql(dataframes, conn):
    # import the dataframes into the database using the function
    for dataframe in dataframes:
        if dataframe is not None:
            import_df_to_sqlite(dataframe, 'val', conn)

In [5]:
def get_links(matches_path):
    driver = webdriver.Chrome(executable_path=chromedriver_path)


    # navigate to a website
    driver.get(matches_path)

    anchors = driver.find_elements(by=By.CSS_SELECTOR, value='a')

    links = []
    # Iterate over each anchor element and extract the href attribute
    for anchor in anchors:
        href = anchor.get_attribute('href')
        links.append(href)

    return links[14:-10]

In [6]:
def get_data_and_upload(link):
    driver.get(link)


    try:
        more_than_one_map = driver.find_element(By.CLASS_NAME, value='vm-stats-gamesnav.noselect  ')
    except NoSuchElementException:
        print('only one map')
        more_than_one_map = None
    dataframes = []

    if more_than_one_map:
        map_buttons_1 = driver.find_element(By.XPATH,
                                            '//*[@id="wrapper"]/div[1]/div[3]/div[6]/div/div[1]/div[2]/div/div[2]')
        map_buttons_2 = driver.find_element(By.XPATH,
                                            '//*[@id="wrapper"]/div[1]/div[3]/div[6]/div/div[1]/div[2]/div/div[3]')
        map_buttons_3 = driver.find_element(By.XPATH,
                                            '//*[@id="wrapper"]/div[1]/div[3]/div[6]/div/div[1]/div[2]/div/div[4]')
        print(map_buttons_1.text, map_buttons_2.text, map_buttons_3.text)
        map_buttons = [map_buttons_1, map_buttons_2, map_buttons_3]

        for map, button in enumerate(map_buttons):
            if 'N/A' not in button.text:
                button.click()
                df = get_data(driver, map + 1, map_buttons[map].text.split()[1])
                dataframes.append(df)


    else:
        df = get_data(driver, 1, driver.find_element(By.CLASS_NAME, value='map').text.split('\n')[0])
        dataframes.append(df)
    ### Add dataframes into SQLite database
    dataframes_to_sql(dataframes, conn)

### GET LINKS TO MATCH RESULTS

### Connect to DB, set paths urls

In [7]:
# create a connection to the SQLite database
conn = sqlite3.connect('valorant.db')

matches_path = 'https://www.vlr.gg/matches/results'
chromedriver_path = r'C:\Users\work\Sportsbetting\Valorant\chromedriver.exe'

### get relevant match links

In [8]:
driver = webdriver.Chrome(executable_path=chromedriver_path)
# # navigate to a website
driver.get(matches_path)

#
# anchors = driver.find_elements(by=By.CSS_SELECTOR, value='a')
#
# links = []
# # Iterate over each anchor element and extract the href attribute
# for anchor in anchors:
#     href = anchor.get_attribute('href')
#     links.append(href)
#
# links = links[14:-10]

C:\Users\work\AppData\Local\Temp\ipykernel_32876\863616162.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=chromedriver_path)


### Add dataframes into SQLite database

In [9]:
first_results_page = '/?page=2'

In [10]:
# links = get_links(matches_path + first_results_page)

In [11]:
links = []

In [12]:
for x in range(2,100):
    links = links + get_links(matches_path + first_results_page[:-1] + str(x))

C:\Users\work\AppData\Local\Temp\ipykernel_32876\1054020278.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=chromedriver_path)


In [13]:
cleaned_links = []
for link in links:
    if len(link) < (len(matches_path) + 10):
        pass
    else:
        cleaned_links.append(link)


In [14]:
cleaned_links

['https://www.vlr.gg/193286/koopa-troopas-vs-xyz-challengers-league-oceania-split-2-group-stage-w7',
 'https://www.vlr.gg/188336/alter-ego-vs-hike-digital-esports-challengers-league-indonesia-split-2-r7',
 'https://www.vlr.gg/204440/attacking-soul-esports-vs-edward-gaming-fgc-valorant-invitational-2023-act-2-ubsf',
 'https://www.vlr.gg/209026/maru-gaming-vs-nongshim-redforce-world-cyber-games-challengers-league-korea-split-2-ubsf',
 'https://www.vlr.gg/193285/funcrew-vs-mammoth-esc-challengers-league-oceania-split-2-group-stage-w7',
 'https://www.vlr.gg/188335/le-crapaud-vs-arf-team-challengers-league-indonesia-split-2-r7',
 'https://www.vlr.gg/209025/dplus-esports-vs-gwangju-shadow-world-cyber-games-challengers-league-korea-split-2-ubsf',
 'https://www.vlr.gg/196162/shopify-rebellion-vs-turtle-troop-challengers-league-north-america-w10',
 'https://www.vlr.gg/193576/red-canids-vs-the-union-challengers-league-brazil-split-2-w7',
 'https://www.vlr.gg/202368/ebro-gaming-vs-furious-gaming-

In [17]:
conn = sqlite3.connect('valorant.db')
for link in cleaned_links:
    try:
        get_data_and_upload(link)
        time.sleep(2.3 + random.random())
    except Exception as n:
        print('match forfeited')
        print(n)



conn.close()

1 Haven 2 Ascent 3 Lotus


C:\Users\work\AppData\Local\Temp\ipykernel_32876\1443927380.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['rounds_won'].iloc[5:] = away_rounds_won
C:\Users\work\AppData\Local\Temp\ipykernel_32876\1443927380.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['rounds_won'].iloc[5:] = away_rounds_won
C:\Users\work\AppData\Local\Temp\ipykernel_32876\1443927380.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['round

1 Fracture 2 Haven 3 Bind


C:\Users\work\AppData\Local\Temp\ipykernel_32876\1443927380.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['rounds_won'].iloc[5:] = away_rounds_won
C:\Users\work\AppData\Local\Temp\ipykernel_32876\1443927380.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['rounds_won'].iloc[5:] = away_rounds_won


map 3 link not working
1 Bind 2 Fracture 3 Haven


C:\Users\work\AppData\Local\Temp\ipykernel_32876\1443927380.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['rounds_won'].iloc[5:] = away_rounds_won
C:\Users\work\AppData\Local\Temp\ipykernel_32876\1443927380.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['rounds_won'].iloc[5:] = away_rounds_won


map 3 link not working
1 Haven 2 Ascent 3 Lotus


C:\Users\work\AppData\Local\Temp\ipykernel_32876\1443927380.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['rounds_won'].iloc[5:] = away_rounds_won
C:\Users\work\AppData\Local\Temp\ipykernel_32876\1443927380.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['rounds_won'].iloc[5:] = away_rounds_won
C:\Users\work\AppData\Local\Temp\ipykernel_32876\1443927380.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['round

In [16]:
conn.close()